<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/TweetEval/TweetEval_EmojiTweetsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 130kB/s 
     |████████████████████████████████| 6.8MB 22.3MB/s 
     |████████████████████████████████| 983kB 56.3MB/s 
     |████████████████████████████████| 266kB 53.2MB/s 
     |████████████████████████████████| 1.9MB 38.9MB/s 
     |████████████████████████████████| 1.2MB 47.5MB/s 
     |████████████████████████████████| 471kB 52.7MB/s 
     |████████████████████████████████| 3.3MB 51.4MB/s 
     |████████████████████████████████| 870kB 48.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=2399484315f4084476315c8ceaa1eb98d5e2f15f79cfed6049c39855d7411adb
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=33da85fb521fcd8035fa8dcb2fa7f833a9569f74f0a5ad79e65fb73bef093f83
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
import re
import requests
import sklearn.metrics

In [3]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/test_labels.txt"
VOCAB_SIZE=2000

In [4]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

114435

In [5]:
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()
val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()
test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

# Labels
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()
val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()
test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()


# Labels
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))
test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))

In [6]:
print('Samples in Training set : ',len(labels),', Validation set : ', len(val_labels),', Test set : ', len(test_labels))

Samples in Training set :  45000 , Validation set :  5000 , Test set :  50000


In [7]:
for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    tweets[i]=re.sub('@user', '', tweets[i])
    tweets[i]=re.sub('#+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])
    val_tweets[i]=re.sub('#+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])
    test_tweets[i]=re.sub('#+', '', test_tweets[i])

In [8]:
for i in range(len(tweets)):
  if tweets[i].find('#[a-zA-Z]+')!=-1:
    tweets[i]=re.sub('#[a-zA-Z]+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('#[a-zA-Z]+')!=-1:
    val_tweets[i]=re.sub('#[a-zA-Z]+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('#[a-zA-Z]+')!=-1:
    test_tweets[i]=re.sub('#[a-zA-Z]+', '', test_tweets[i])

In [9]:
for i in range(len(tweets)):
  if tweets[i].find('\n')!=-1:
    tweets[i]=re.sub('\n', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('\n')!=-1:
    val_tweets[i]=re.sub('\n', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('\n')!=-1:
    test_tweets[i]=re.sub('\n', '', test_tweets[i])

# Unnecessary dots
p='\.\.\.|\.\.'

for i in range(len(tweets)):
  tweets[i]=re.sub(p, '', tweets[i])
  tweets[i]=tweets[i].lower()

for i in range(len(val_tweets)):
  val_tweets[i]=re.sub(p, '', val_tweets[i])
  val_tweets[i]=val_tweets[i].lower()

for i in range(len(test_tweets)):
  test_tweets[i]=re.sub(p, '', test_tweets[i])
  test_tweets[i]=test_tweets[i].lower()

In [10]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
train_df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [11]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=train_df,
                                                                   text_column = 'TWEET',
                                                                   label_columns = 'CATEGORY',
                                                                   val_df = val_df,
                                                                   maxlen = 65,
                                                                   preprocess_mode = 'bert')

['CATEGORY_0', 'CATEGORY_1', 'CATEGORY_2', 'CATEGORY_3', 'CATEGORY_4', 'CATEGORY_5', 'CATEGORY_6', 'CATEGORY_7', 'CATEGORY_8', 'CATEGORY_9', 'CATEGORY_10', 'CATEGORY_11', 'CATEGORY_12', 'CATEGORY_13', 'CATEGORY_14', 'CATEGORY_15', 'CATEGORY_16', 'CATEGORY_17', 'CATEGORY_18', 'CATEGORY_19']
   CATEGORY_0  CATEGORY_1  CATEGORY_2  ...  CATEGORY_17  CATEGORY_18  CATEGORY_19
1         0.0         0.0         0.0  ...          0.0          0.0          0.0
2         0.0         0.0         0.0  ...          0.0          0.0          1.0
3         1.0         0.0         0.0  ...          0.0          0.0          0.0
4         1.0         0.0         0.0  ...          0.0          0.0          0.0
5         0.0         0.0         1.0  ...          0.0          0.0          0.0

[5 rows x 20 columns]
['CATEGORY_0', 'CATEGORY_1', 'CATEGORY_2', 'CATEGORY_3', 'CATEGORY_4', 'CATEGORY_5', 'CATEGORY_6', 'CATEGORY_7', 'CATEGORY_8', 'CATEGORY_9', 'CATEGORY_10', 'CATEGORY_11', 'CATEGORY_12', 'CATEGOR

Is Multi-Label? False
preprocessing test...
language: en


In [12]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 65
done.


In [15]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 5)

In [16]:
learner.fit_onecycle(lr = 2e-5, epochs = 2)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert')



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
9000/9000 [==============================] - 2547s 282ms/step - loss: 2.3749 - accuracy: 0.3025 - val_loss: 2.2062 - val_accuracy: 0.3408
Epoch 2/2
9000/9000 [==============================] - 2511s 279ms/step - loss: 1.9894 - accuracy: 0.3973 - val_loss: 2.1065 - val_accuracy: 0.3640


In [17]:
data = test_df['TWEET'].tolist()

In [18]:
bert_pred=predictor.predict(data)

In [19]:
bert_pred[:20]

['CATEGORY_0',
 'CATEGORY_10',
 'CATEGORY_7',
 'CATEGORY_1',
 'CATEGORY_1',
 'CATEGORY_17',
 'CATEGORY_4',
 'CATEGORY_10',
 'CATEGORY_12',
 'CATEGORY_10',
 'CATEGORY_0',
 'CATEGORY_12',
 'CATEGORY_0',
 'CATEGORY_2',
 'CATEGORY_6',
 'CATEGORY_4',
 'CATEGORY_0',
 'CATEGORY_1',
 'CATEGORY_0',
 'CATEGORY_2']

In [20]:
y_true=np.array(test_df['CATEGORY'].tolist())

In [21]:
y_true[:20]

array([ 2, 10,  6,  1, 16, 17,  4, 10, 12, 18,  0,  1,  5,  7,  7,  4,  0,
        7,  0, 10])

In [22]:
res_bert=[]
for i in range(len(bert_pred)):
  if bert_pred[i]=='CATEGORY_0':
    res_bert.append(0)
  elif bert_pred[i]=='CATEGORY_1':
    res_bert.append(1)
  elif bert_pred[i]=='CATEGORY_2':
    res_bert.append(2)
  elif bert_pred[i]=='CATEGORY_3':
    res_bert.append(3)
  elif bert_pred[i]=='CATEGORY_4':
    res_bert.append(4)
  elif bert_pred[i]=='CATEGORY_5':
    res_bert.append(5)
  elif bert_pred[i]=='CATEGORY_6':
    res_bert.append(6)
  elif bert_pred[i]=='CATEGORY_7':
    res_bert.append(7)
  elif bert_pred[i]=='CATEGORY_8':
    res_bert.append(8)
  elif bert_pred[i]=='CATEGORY_9':
    res_bert.append(9)
  elif bert_pred[i]=='CATEGORY_10':
    res_bert.append(10)
  elif bert_pred[i]=='CATEGORY_11':
    res_bert.append(11)
  elif bert_pred[i]=='CATEGORY_12':
    res_bert.append(12)
  elif bert_pred[i]=='CATEGORY_13':
    res_bert.append(13)
  elif bert_pred[i]=='CATEGORY_14':
    res_bert.append(14)
  elif bert_pred[i]=='CATEGORY_15':
    res_bert.append(15)
  elif bert_pred[i]=='CATEGORY_16':
    res_bert.append(16)
  elif bert_pred[i]=='CATEGORY_17':
    res_bert.append(17)
  elif bert_pred[i]=='CATEGORY_18':
    res_bert.append(18)
  else:
    res_bert.append(19)

In [23]:
res_bert=np.array(res_bert)
np.mean(res_bert==y_true)

0.37096

In [24]:
sklearn.metrics.f1_score(y_true, res_bert, average='macro')

0.2546177923587214